**Importing required modules**
* `requests` used for importing the downloaded zip file
* `zipfile` for opening the zip file
* `pandas` as datatype

In [6]:
import requests
import zipfile
import pandas as pd

**Importing file and addressing**

In [7]:
streetdataZIP_url = 'https://drive.google.com/uc?export=download&id=16c8A7GCe9VnTLYCZI9v15K64VCBoNnYf'
response = requests.get(streetdataZIP_url)
open("streetdata.zip", "wb").write(response.content)
zf = zipfile.ZipFile('streetdata.zip')
zf.extractall()
streetname_df = pd.read_csv('all_data_mapping_diversity/allstreets_allcities_aug-23.csv')
print(streetname_df.head())

   gisco_id lau_name country         street_name  named_after_n label  \
0  AT_90001     Wien      AT         "Am Wasser"            1.0   NaN   
1  AT_90001     Wien      AT                   1            1.0   NaN   
2  AT_90001     Wien      AT  1. Haidequerstraße            1.0   NaN   
3  AT_90001     Wien      AT       1. Molostraße            1.0   NaN   
4  AT_90001     Wien      AT           1. Straße            1.0   NaN   

  named_after_id instance_of_label  person description  ...  \
0            NaN               NaN     0.0         NaN  ...   
1            NaN               NaN     0.0         NaN  ...   
2            NaN               NaN     0.0         NaN  ...   
3            NaN               NaN     0.0         NaN  ...   
4            NaN               NaN     0.0         NaN  ...   

  place_of_birth_label place_of_death_label country_of_citizenship_label  \
0                  NaN                  NaN                          NaN   
1                  NaN        